# Home assignment 02: Laplace distribution

Today your goal is to build a class for Laplace distribution. The part of the notebook copies the one from the practice session.

## Loading data

In [93]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

In [94]:
matplotlib.rcParams['font.size'] = 11

First to load dataset we're going to use [`sklearn`](https://scikit-learn.org/stable/) package which we will extensively use during the whole course.

`sklearn` implement most of classical and frequently used algorithms in Machine Learning. Also it provides [User Guide](https://scikit-learn.org/stable/user_guide.html) describing principles of every bunch of algorithms implemented.

As an entry point to main `sklearn`'s concepts we recommend [getting started tutorial](https://scikit-learn.org/stable/getting_started.html) (check it out yourself). [Further tutorials](https://scikit-learn.org/stable/tutorial/index.html) can also be handy to develop your skills.

In [95]:
dataset = datasets.load_iris()

print(dataset.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

If you aren't familiar with Iris dataset - take a minute to read description above (as always [more info about it in Wikipedia](https://en.wikipedia.org/wiki/Iris_flower_data_set)).

__TL;DR__ 150 objects equally distributed over 3 classes each described with 4 continuous features

Just pretty table to look at:

In [96]:
ext_target = dataset.target[:, None]
pd.DataFrame(
    np.concatenate((dataset.data, ext_target, dataset.target_names[ext_target]), axis=1),
    columns=dataset.feature_names + ['target label', 'target name'],
)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target label,target name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


Now give distinct names to the data we will use

In [97]:
features = dataset.data
target = dataset.target

features.shape, target.shape

((150, 4), (150,))

__Please, remember!!!__

Anywhere in our course we have an agreement to shape design matrix (named `features` in code above) as 

`(#number_of_items, #number_of_features)` if not stated explicitly.

## Distribution implementation

Let's implement class taking list of feature values, estimating Laplace distribution params and able to give probability density of any given feature value.

The file downloaded below contains the template for your class.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23s_dd_ml/homeworks/hw02_laplace/distribution.py

Denote the Laplace distribution $\mathcal{L}(\mu, b)$ PDF, where $\mu$ stand for location (loc), and $b$ stands for scale:
$$
f(x|\mu, b) = \frac{1}{2b}\exp(-\frac{|x - \mu|}{b})
$$
Let's implement the `LaplaceDistribution` class. (Of course in practice one could always use something like `scipy.stats.laplace`).

Please note, that making computations with log probabilities is more stable.


#### Description [from Wikipedia](https://en.wikipedia.org/wiki/Laplace_distribution#Statistical_inference):

Given $n$ independent and identically distributed samples $x_1, x_2, ..., x_n$, the maximum likelihood (MLE) estimator of $\mu$ is the sample median:
$$
\hat{\mu} = \mathrm{median}(x).
$$



The MLE estimator $b$ is the mean absolute deviation from the median
$$
\hat{b} = \frac{1}{n} \sum_{i = 1}^{n} |x_i - \hat{\mu}|.$$

revealing a link between the Laplace distribution and least absolute deviations.
A correction for small samples can be applied as follows:
$\hat{b}^* = \hat{b} \cdot n/(n-2)$


In [ ]:
# Run some setup code for this notebook.
import random
import numpy as np
import matplotlib.pyplot as plt

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# This dirty hack might help if the autoreload has failed for some reason
try:
    del LaplaceDistribution
except:
    pass

from distribution import LaplaceDistribution

### Distribution parameters check

In [115]:
import scipy

loc0, scale0 = scipy.stats.laplace.fit(features[:, 0])
loc1, scale1 = scipy.stats.laplace.fit(features[:, 1])

# 1d case
my_distr_1 = LaplaceDistribution(features[:, 0])

# check the 1d median (loc parameter)
assert np.allclose(my_distr_1.loc, loc0), '1d distribution median error'
# check the 1d scale (loc parameter)
assert np.allclose(my_distr_1.scale, scale0), '1d distribution scale error'


# 2d case
my_distr_2 = LaplaceDistribution(features[:, :2])

# check the 2d median (loc parameter)
assert np.allclose(my_distr_2.loc, np.array([loc0, loc1])), '2d distribution median error'
# check the 2d median (loc parameter)
assert np.allclose(my_distr_2.scale, np.array([scale0, scale1])), '2d distribution scale error'



print('Seems fine!')

Seems fine!


### Distribution logpdf check

In [120]:
_test = scipy.stats.laplace(loc=[loc0, loc1], scale=[scale0, scale1])
    
assert np.allclose(
    my_distr_2.logpdf(features[:5, :2]),
    _test.logpdf(features[:5, :2])
), 'Logpdfs do not match scipy results!'
print('Seems fine!')

Seems fine!


Congratulations! Please, paste the `LaplaceDistribution` class code into the py file and submit it to the contest system.